In [23]:
import json 
import pandas as pd
import numpy as np
import pickle
import re
import ast
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob

In [2]:
rows = []
with open("australian_user_reviews.json", encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

In [3]:
rows

[{'user_id': '76561197970982479',
  'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
  'reviews': [{'funny': '',
    'posted': 'Posted November 5, 2011.',
    'last_edited': '',
    'item_id': '1250',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
   {'funny': '',
    'posted': 'Posted July 15, 2011.',
    'last_edited': '',
    'item_id': '22200',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': "It's unique and worth a playthrough."},
   {'funny': '',
    'posted': 'Posted April 21, 2011.',
    'last_edited': '',
    'item_id': '43110',
    'helpful': 'No ratings yet',
    'recommend': True,
    'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at t

In [4]:
reviews = pd.json_normalize(rows, record_path=['reviews'], meta=['user_id','user_url'] )

In [6]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [7]:
reviews.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...


BUSCANDO DUPLICADOS DE EXISTIR SE PROCEDERA A ELIMINAR

In [9]:
reviews.duplicated().sum()

874

In [11]:
reviews = reviews.drop_duplicates()
reviews.reset_index(drop=True, inplace=True)

NORMALIZANDO LA COLUMNA POSTED

In [15]:
reviews['posted'] = reviews['posted'].str.replace('Posted', '')


from datetime import datetime

def getNormalizeDate(text):   
   
    text = str.lstrip(str.rstrip(text))
    
    if len(text)>4 and "," in text:
         
        formats = ["%B %d %Y", "%d %B %Y", "%b %d %Y", "%m/%d/%Y", "%m %d %Y","%m/%d"]
        text = text.replace(".", "").replace(",", "")
        
        for format in formats:
            return datetime.strptime(text, format).strftime('%Y-%m-%d')

In [ ]:
from datetime import datetime
reviews = pd.DataFrame({'posted': ['Posted on September 15, 2023', 'Posted on 12/31/2022', 'Posted on Feb 10, 2021']})
def getNormalizedDate(text):
    try:
        text = text.replace('Posted', '').strip().replace(',', '').replace('.', '')
        return pd.to_datetime(text, format='%B %d %Y', errors='raise').strftime('%Y-%m-%d')
    except ValueError:
        return pd.NaT 
reviews['posted'] = reviews['posted'].apply(getNormalizedDate)

In [16]:
reviews['posted'] = reviews['posted'].astype(str).apply(getNormalizeDate)

In [18]:
reviews.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...


BUSCANDO NULOS Y FALTANTES

In [19]:
reviews.isnull().sum()

funny             0
posted         9933
last_edited       0
item_id           0
helpful           0
recommend         0
review            0
user_id           0
user_url          0
dtype: int64

In [20]:
reviews.isna().sum()

funny             0
posted         9933
last_edited       0
item_id           0
helpful           0
recommend         0
review            0
user_id           0
user_url          0
dtype: int64

In [21]:
reviews['posted'] = reviews['posted'].replace(np.nan, '')

In [22]:
reviews.tail(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
58429,,,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
58430,,,,440,No ratings yet,True,so much fun :D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


ANALISIS DE SENTIMIENTO CREACION DE LA COLUMNA SENTIMENT_ANALYSIS Y ELIMINACION DE REVIEW SEGUN INDICADA EL README DEL PROYECTO

In [24]:
def analyze_sentiment(text):
    if isinstance(text, str):
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        if polarity < 0:
            return 0  # Malo
        elif polarity == 0:
            return 1  # Neutral
        else:
            return 2  # Positivo
    else:
        return 1  # Valor predeterminado si no hay reseña escrita


reviews['sentiment_analysis'] = reviews['review'].apply(analyze_sentiment)




reviews.drop(columns=['review'], inplace=True)

reviews


,funny,posted,last_edited,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,,2011-11-05,,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
1,,2011-07-15,,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,,2011-04-21,,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
3,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,2
4,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,0
...,...,...,...,...,...,...,...,...,...
58426,,,,70,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
58427,,,,362890,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
58428,1 person found this review funny,,,273110,1 of 2 people (50%) found this review helpful,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2
58429,,,,730,No ratings yet,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2


In [25]:
reviews.to_parquet('ETLreviews.parquet', engine= 'pyarrow')